#tweetdeleterapp


### ghostokaamiii
github

"""
Twitter Tweet Deletion App - Jupyter Notebook Compatible
A Python application to bulk delete tweets with filtering options.
"""



In [ ]:
#!/usr/bin/env python3
"""
Simple Twitter Tweet Deletion App - GUI Version
A simplified Python GUI application to bulk delete tweets.
"""

import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, filedialog
import tweepy
import time
import json
import threading
from datetime import datetime, timedelta
from typing import List, Optional, Dict
import os
from dataclasses import dataclass


@dataclass
class DeleteConfig:
    """Configuration for tweet deletion parameters"""
    older_than_days: Optional[int] = None
    contains_text: Optional[str] = None
    exclude_text: Optional[str] = None
    max_likes: Optional[int] = None
    exclude_replies: bool = False
    exclude_retweets: bool = False
    dry_run: bool = True


class TwitterDeleter:
    def __init__(self, bearer_token: str, api_key: str, api_secret: str, 
                 access_token: str, access_token_secret: str):
        """Initialize Twitter API client"""
        self.client = tweepy.Client(
            bearer_token=bearer_token,
            consumer_key=api_key,
            consumer_secret=api_secret,
            access_token=access_token,
            access_token_secret=access_token_secret,
            wait_on_rate_limit=True
        )
        
        self.deleted_count = 0
        self.skipped_count = 0
        
    def get_user_tweets(self, user_id: str, max_results: int = 100, callback=None) -> List[tweepy.Tweet]:
        """Fetch user's tweets with optional progress callback"""
        tweets = []
        pagination_token = None
        
        while True:
            try:
                response = self.client.get_users_tweets(
                    id=user_id,
                    max_results=min(max_results, 100),
                    pagination_token=pagination_token,
                    tweet_fields=['created_at', 'public_metrics', 'context_annotations', 'conversation_id'],
                    exclude=None
                )
                
                if not response.data:
                    break
                    
                tweets.extend(response.data)
                if callback:
                    callback(f"Fetched {len(response.data)} tweets (Total: {len(tweets)})")
                
                if 'next_token' not in response.meta:
                    break
                    
                pagination_token = response.meta['next_token']
                time.sleep(1)
                
            except Exception as e:
                if callback:
                    callback(f"Error fetching tweets: {e}")
                break
                
        return tweets
    
    def should_delete_tweet(self, tweet: tweepy.Tweet, config: DeleteConfig) -> bool:
        """Determine if a tweet should be deleted based on configuration"""
        # Date filtering
        if config.older_than_days:
            cutoff_date = datetime.now() - timedelta(days=config.older_than_days)
            if tweet.created_at > cutoff_date:
                return False
        
        # Text filtering
        tweet_text = tweet.text.lower()
        
        if config.contains_text:
            if config.contains_text.lower() not in tweet_text:
                return False
                
        if config.exclude_text:
            if config.exclude_text.lower() in tweet_text:
                return False
        
        # Engagement filtering
        if config.max_likes:
            metrics = tweet.public_metrics
            if metrics['like_count'] > config.max_likes:
                return False
        
        # Reply/retweet filtering
        if config.exclude_replies and tweet.in_reply_to_user_id:
            return False
            
        if config.exclude_retweets and tweet.text.startswith('RT @'):
            return False
            
        return True
    
    def delete_tweet(self, tweet_id: str) -> bool:
        """Delete a single tweet"""
        try:
            self.client.delete_tweet(tweet_id)
            return True
        except Exception:
            return False
    
    def bulk_delete(self, config: DeleteConfig, progress_callback=None) -> Dict[str, int]:
        """Perform bulk tweet deletion with progress callback"""
        try:
            user = self.client.get_me()
            if progress_callback:
                progress_callback(f"Authenticated as: @{user.data.username}")
            user_id = user.data.id
        except Exception as e:
            if progress_callback:
                progress_callback(f"Authentication failed: {e}")
            return {'deleted': 0, 'skipped': 0, 'errors': 1}
        
        # Fetch tweets
        tweets = self.get_user_tweets(user_id, max_results=100, callback=progress_callback)
        if progress_callback:
            progress_callback(f"Found {len(tweets)} total tweets")
        
        if not tweets:
            if progress_callback:
                progress_callback("No tweets found")
            return {'deleted': 0, 'skipped': 0, 'errors': 0}
        
        # Filter tweets
        tweets_to_delete = []
        for tweet in tweets:
            if self.should_delete_tweet(tweet, config):
                tweets_to_delete.append(tweet)
        
        if progress_callback:
            progress_callback(f"Found {len(tweets_to_delete)} tweets matching criteria")
        
        if not tweets_to_delete:
            return {'deleted': 0, 'skipped': len(tweets), 'errors': 0}
        
        if config.dry_run:
            if progress_callback:
                progress_callback(f"DRY RUN: Would delete {len(tweets_to_delete)} tweets")
                # Show preview of tweets
                progress_callback("Preview of tweets to delete:")
                for i, tweet in enumerate(tweets_to_delete[:5]):
                    progress_callback(f"{i+1}. [{tweet.created_at}] {tweet.text[:100]}...")
                if len(tweets_to_delete) > 5:
                    progress_callback(f"... and {len(tweets_to_delete) - 5} more tweets")
            return {'deleted': 0, 'skipped': len(tweets), 'dry_run': len(tweets_to_delete)}
        
        # Delete tweets
        deleted_count = 0
        error_count = 0
        
        for i, tweet in enumerate(tweets_to_delete, 1):
            if progress_callback:
                progress_callback(f"Deleting tweet {i}/{len(tweets_to_delete)}: {tweet.text[:50]}...")
            
            if self.delete_tweet(tweet.id):
                deleted_count += 1
            else:
                error_count += 1
            
            if i % 50 == 0:
                if progress_callback:
                    progress_callback("Pausing for rate limits...")
                time.sleep(60)
        
        if progress_callback:
            progress_callback(f"Complete! Deleted: {deleted_count}, Errors: {error_count}")
        
        return {
            'deleted': deleted_count,
            'skipped': len(tweets) - len(tweets_to_delete),
            'errors': error_count
        }


class SimpleTwitterDeleterGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Twitter Tweet Deleter")
        self.root.geometry("700x800")
        
        self.deleter = None
        self.config_file = "twitter_config.json"
        
        self.setup_gui()
        self.load_config()
        
    def setup_gui(self):
        """Create the GUI layout"""
        # Main frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill='both', expand=True)
        
        # Title
        title = ttk.Label(main_frame, text="Twitter Tweet Deleter", font=('Arial', 16, 'bold'))
        title.pack(pady=(0, 20))
        
        # API Configuration Section
        self.setup_api_section(main_frame)
        
        # Deletion Settings Section
        self.setup_deletion_section(main_frame)
        
        # Action Buttons
        self.setup_action_section(main_frame)
        
        # Results Section
        self.setup_results_section(main_frame)
        
    def setup_api_section(self, parent):
        """Setup API configuration section"""
        api_frame = ttk.LabelFrame(parent, text="API Configuration", padding="10")
        api_frame.pack(fill='x', pady=(0, 10))
        
        # Simplified credential input
        self.bearer_token_var = tk.StringVar()
        self.api_key_var = tk.StringVar()
        self.api_secret_var = tk.StringVar()
        self.access_token_var = tk.StringVar()
        self.access_token_secret_var = tk.StringVar()
        
        ttk.Label(api_frame, text="Bearer Token:").grid(row=0, column=0, sticky='w', pady=2)
        ttk.Entry(api_frame, textvariable=self.bearer_token_var, width=60, show='*').grid(row=0, column=1, pady=2)
        
        ttk.Label(api_frame, text="API Key:").grid(row=1, column=0, sticky='w', pady=2)
        ttk.Entry(api_frame, textvariable=self.api_key_var, width=60, show='*').grid(row=1, column=1, pady=2)
        
        ttk.Label(api_frame, text="API Secret:").grid(row=2, column=0, sticky='w', pady=2)
        ttk.Entry(api_frame, textvariable=self.api_secret_var, width=60, show='*').grid(row=2, column=1, pady=2)
        
        ttk.Label(api_frame, text="Access Token:").grid(row=3, column=0, sticky='w', pady=2)
        ttk.Entry(api_frame, textvariable=self.access_token_var, width=60, show='*').grid(row=3, column=1, pady=2)
        
        ttk.Label(api_frame, text="Access Token Secret:").grid(row=4, column=0, sticky='w', pady=2)
        ttk.Entry(api_frame, textvariable=self.access_token_secret_var, width=60, show='*').grid(row=4, column=1, pady=2)
        
        # API buttons
        api_buttons = ttk.Frame(api_frame)
        api_buttons.grid(row=5, column=0, columnspan=2, pady=10)
        
        ttk.Button(api_buttons, text="Save Config", command=self.save_config).pack(side='left', padx=5)
        ttk.Button(api_buttons, text="Load Config", command=self.load_config).pack(side='left', padx=5)
        ttk.Button(api_buttons, text="Test Connection", command=self.test_connection).pack(side='left', padx=5)
        
        # Status
        self.status_var = tk.StringVar(value="Load or enter your API credentials")
        ttk.Label(api_frame, textvariable=self.status_var, foreground='blue').grid(row=6, column=0, columnspan=2, pady=5)
        
    def setup_deletion_section(self, parent):
        """Setup deletion settings section"""
        delete_frame = ttk.LabelFrame(parent, text="Deletion Settings", padding="10")
        delete_frame.pack(fill='x', pady=(0, 10))
        
        # Simple filters
        self.older_than_var = tk.StringVar()
        self.contains_var = tk.StringVar()
        self.exclude_var = tk.StringVar()
        self.max_likes_var = tk.StringVar()
        
        ttk.Label(delete_frame, text="Delete tweets older than (days):").grid(row=0, column=0, sticky='w', pady=2)
        ttk.Entry(delete_frame, textvariable=self.older_than_var, width=10).grid(row=0, column=1, sticky='w', padx=10, pady=2)
        
        ttk.Label(delete_frame, text="Only delete tweets containing:").grid(row=1, column=0, sticky='w', pady=2)
        ttk.Entry(delete_frame, textvariable=self.contains_var, width=40).grid(row=1, column=1, sticky='w', padx=10, pady=2)
        
        ttk.Label(delete_frame, text="Exclude tweets containing:").grid(row=2, column=0, sticky='w', pady=2)
        ttk.Entry(delete_frame, textvariable=self.exclude_var, width=40).grid(row=2, column=1, sticky='w', padx=10, pady=2)
        
        ttk.Label(delete_frame, text="Max likes (delete if ≤):").grid(row=3, column=0, sticky='w', pady=2)
        ttk.Entry(delete_frame, textvariable=self.max_likes_var, width=10).grid(row=3, column=1, sticky='w', padx=10, pady=2)
        
        # Checkboxes
        checkbox_frame = ttk.Frame(delete_frame)
        checkbox_frame.grid(row=4, column=0, columnspan=2, sticky='w', pady=10)
        
        self.exclude_replies_var = tk.BooleanVar()
        self.exclude_retweets_var = tk.BooleanVar(value=True)
        self.dry_run_var = tk.BooleanVar(value=True)
        
        ttk.Checkbutton(checkbox_frame, text="Exclude replies", variable=self.exclude_replies_var).pack(anchor='w')
        ttk.Checkbutton(checkbox_frame, text="Exclude retweets", variable=self.exclude_retweets_var).pack(anchor='w')
        ttk.Checkbutton(checkbox_frame, text="Dry run (preview only)", variable=self.dry_run_var).pack(anchor='w', pady=(5, 0))
        
        # Warning
        warning = ttk.Label(delete_frame, text="⚠️ Always test with dry run first!", foreground='red')
        warning.grid(row=5, column=0, columnspan=2, pady=5)
        
    def setup_action_section(self, parent):
        """Setup action buttons section"""
        action_frame = ttk.Frame(parent)
        action_frame.pack(fill='x', pady=(0, 10))
        
        ttk.Button(action_frame, text="Preview Deletion", command=self.preview_deletion).pack(side='left', padx=5)
        ttk.Button(action_frame, text="Start Deletion", command=self.start_deletion).pack(side='left', padx=5)
        ttk.Button(action_frame, text="Load Preset", command=self.load_preset).pack(side='left', padx=5)
        ttk.Button(action_frame, text="Save Preset", command=self.save_preset).pack(side='left', padx=5)
        
    def setup_results_section(self, parent):
        """Setup results display section"""
        results_frame = ttk.LabelFrame(parent, text="Results", padding="10")
        results_frame.pack(fill='both', expand=True)
        
        # Progress
        self.progress_var = tk.StringVar(value="Ready")
        ttk.Label(results_frame, textvariable=self.progress_var).pack(anchor='w', pady=(0, 5))
        
        self.progress_bar = ttk.Progressbar(results_frame, mode='indeterminate')
        self.progress_bar.pack(fill='x', pady=(0, 10))
        
        # Results text
        self.results_text = scrolledtext.ScrolledText(results_frame, height=15, width=80)
        self.results_text.pack(fill='both', expand=True)
        
        # Clear button
        ttk.Button(results_frame, text="Clear Results", command=self.clear_results).pack(pady=(5, 0))
        
    def get_int_or_none(self, var):
        """Helper to convert string var to int or None"""
        try:
            value = var.get().strip()
            return int(value) if value else None
        except ValueError:
            return None
            
    def get_string_or_none(self, var):
        """Helper to convert string var to string or None"""
        value = var.get().strip()
        return value if value else None
        
    def create_delete_config(self):
        """Create DeleteConfig from GUI inputs"""
        return DeleteConfig(
            older_than_days=self.get_int_or_none(self.older_than_var),
            contains_text=self.get_string_or_none(self.contains_var),
            exclude_text=self.get_string_or_none(self.exclude_var),
            max_likes=self.get_int_or_none(self.max_likes_var),
            exclude_replies=self.exclude_replies_var.get(),
            exclude_retweets=self.exclude_retweets_var.get(),
            dry_run=self.dry_run_var.get()
        )
        
    def save_config(self):
        """Save API configuration"""
        config = {
            "bearer_token": self.bearer_token_var.get(),
            "api_key": self.api_key_var.get(),
            "api_secret": self.api_secret_var.get(),
            "access_token": self.access_token_var.get(),
            "access_token_secret": self.access_token_secret_var.get()
        }
        
        try:
            with open(self.config_file, 'w') as f:
                json.dump(config, f, indent=2)
            self.status_var.set("Configuration saved successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save configuration: {e}")
            
    def load_config(self):
        """Load API configuration"""
        try:
            with open(self.config_file, 'r') as f:
                config = json.load(f)
                
            self.bearer_token_var.set(config.get("bearer_token", ""))
            self.api_key_var.set(config.get("api_key", ""))
            self.api_secret_var.set(config.get("api_secret", ""))
            self.access_token_var.set(config.get("access_token", ""))
            self.access_token_secret_var.set(config.get("access_token_secret", ""))
            
            self.status_var.set("Configuration loaded successfully!")
        except FileNotFoundError:
            self.status_var.set("No config file found. Please enter your credentials.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load configuration: {e}")
            
    def test_connection(self):
        """Test Twitter API connection"""
        try:
            self.deleter = TwitterDeleter(
                bearer_token=self.bearer_token_var.get(),
                api_key=self.api_key_var.get(),
                api_secret=self.api_secret_var.get(),
                access_token=self.access_token_var.get(),
                access_token_secret=self.access_token_secret_var.get()
            )
            
            user = self.deleter.client.get_me()
            self.status_var.set(f"✓ Connected as @{user.data.username}")
            messagebox.showinfo("Success", f"Successfully connected as @{user.data.username}")
            
        except Exception as e:
            self.status_var.set("✗ Connection failed")
            messagebox.showerror("Connection Error", f"Failed to connect: {e}")
            
    def preview_deletion(self):
        """Preview what would be deleted"""
        if not self.deleter:
            messagebox.showerror("Error", "Please test your API connection first")
            return
            
        config = self.create_delete_config()
        config.dry_run = True
        
        self.run_deletion_in_thread(config)
        
    def start_deletion(self):
        """Start the deletion process"""
        if not self.deleter:
            messagebox.showerror("Error", "Please test your API connection first")
            return
            
        config = self.create_delete_config()
        
        if not config.dry_run:
            response = messagebox.askyesno(
                "Confirm Deletion", 
                "Are you sure you want to delete tweets? This cannot be undone!"
            )
            if not response:
                return
        
        self.run_deletion_in_thread(config)
        
    def run_deletion_in_thread(self, config):
        """Run deletion in a separate thread"""
        self.progress_bar.start()
        self.progress_var.set("Processing...")
        
        def deletion_thread():
            def progress_callback(message):
                self.root.after(0, lambda: self.update_results(message))
                
            try:
                result = self.deleter.bulk_delete(config, progress_callback)
                self.root.after(0, lambda: self.deletion_complete(result))
            except Exception as e:
                self.root.after(0, lambda: self.deletion_error(str(e)))
                
        threading.Thread(target=deletion_thread, daemon=True).start()
        
    def update_results(self, message):
        """Update results display"""
        self.results_text.insert(tk.END, f"{datetime.now().strftime('%H:%M:%S')}: {message}\n")
        self.results_text.see(tk.END)
        self.root.update_idletasks()
        
    def deletion_complete(self, result):
        """Handle deletion completion"""
        self.progress_bar.stop()
        self.progress_var.set("Complete")
        
        summary = f"\n=== SUMMARY ===\n"
        summary += f"Deleted: {result.get('deleted', 0)}\n"
        summary += f"Skipped: {result.get('skipped', 0)}\n"
        summary += f"Errors: {result.get('errors', 0)}\n"
        if 'dry_run' in result:
            summary += f"Would delete: {result['dry_run']}\n"
        summary += "===============\n\n"
        
        self.results_text.insert(tk.END, summary)
        self.results_text.see(tk.END)
        
    def deletion_error(self, error):
        """Handle deletion error"""
        self.progress_bar.stop()
        self.progress_var.set("Error")
        self.update_results(f"ERROR: {error}")
        
    def clear_results(self):
        """Clear results text"""
        self.results_text.delete(1.0, tk.END)
        
    def load_preset(self):
        """Load deletion preset from file"""
        file_path = filedialog.askopenfilename(
            title="Load Preset",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
        )
        
        if file_path:
            try:
                with open(file_path, 'r') as f:
                    preset = json.load(f)
                    
                self.older_than_var.set(str(preset.get('older_than_days', '')))
                self.contains_var.set(preset.get('contains_text', ''))
                self.exclude_var.set(preset.get('exclude_text', ''))
                self.max_likes_var.set(str(preset.get('max_likes', '')))
                self.exclude_replies_var.set(preset.get('exclude_replies', False))
                self.exclude_retweets_var.set(preset.get('exclude_retweets', True))
                self.dry_run_var.set(preset.get('dry_run', True))
                
                messagebox.showinfo("Success", "Preset loaded successfully!")
                
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load preset: {e}")
                
    def save_preset(self):
        """Save current settings as preset"""
        file_path = filedialog.asksaveasfilename(
            title="Save Preset",
            defaultextension=".json",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
        )
        
        if file_path:
            try:
                config = self.create_delete_config()
                preset = {
                    'older_than_days': config.older_than_days,
                    'contains_text': config.contains_text,
                    'exclude_text': config.exclude_text,
                    'max_likes': config.max_likes,
                    'exclude_replies': config.exclude_replies,
                    'exclude_retweets': config.exclude_retweets,
                    'dry_run': config.dry_run
                }
                
                with open(file_path, 'w') as f:
                    json.dump(preset, f, indent=2)
                    
                messagebox.showinfo("Success", "Preset saved successfully!")
                
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save preset: {e}")


def main():
    root = tk.Tk()
    app = SimpleTwitterDeleterGUI(root)
    root.mainloop()


if __name__ == "__main__":
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_2200\2676539299.py", line 459, in <lambda>
    self.root.after(0, lambda: self.deletion_error(str(e)))
                                                       ^
NameError: cannot access free variable 'e' where it is not associated with a value in enclosing scope
Rate limit exceeded. Sleeping for 834 seconds.
